To do's:
- When there is a larger set of data with weather predictions (after 2021/06/15), remove weather observations and switch to only using weather predictions
- Update list of locations to use in report
- Improve/validate prediction model based on above changes

# Resono 1 week predictions 

Make 1 week-ahead predictions of the visitor counts (hourly visitor counts based on historic data) for all locations (in druktebeeld) or a list of Resono locations. 

Generate a graph with the predictions for in the weekly report for each location. The graph will be automatically saved in the directory that you can set in the arguments section. 

Predictions are stored in a data frame with the following additional columns: 
- **'total_count_predicted'**: predicted total counts (for the next 7 days per location)
- **'data_version'**: version of the data (feature set)
- **'model_version'**: version of the model (type and settings)
- **'predicted_at'**: timestamp of prediction (moment prediction was made)

#### Information for when using this notebook:

Data file needed: 
- Hourly total counts of historic Resono data (retrieved from Resono dashboard)

Current model:
- Linear regression (based on validation with 7 weeks for selection of locations, against baseline model (repeat past week))

Model input current version:
- Past observations Resono data (average past few weeks etc.)
- Periodic data (time of day etc.)
- Stringency Index
- Holiday data
- Vacation data
- Weather data **observations** (< 2021/06/15), and **predictions** (>= 2021/06/15) (temperature, wind speed, global radiation, cloud cover)

### Preparations

Change directory to folder that contains the function files/database credentials in code blocks below.

In [2]:
def install_packages():
    # (Re-)Installs packages.
    
    get_ipython().run_cell_magic('bash', '', 'pip install imblearn\npip install xgboost\npip install mord\npip install psycopg2-binary\npip install workalendar\npip install eli5\n pip install plotly')
    
    import pandas as pd
    pd.set_option('mode.chained_assignment', None)

In [3]:
%%capture
install_packages()

In [4]:
#pip install scikit-learn==0.24.2  # Run if sklearn error 

In [5]:
import os
import pandas as pd
from time import time
from datetime import timedelta

#os.chdir("/home/jovyan/Credentials") # Directory with Azure DB credentials
import env_az

#os.chdir("/home/jovyan/gitops/central_storage_analyses/notebooks_predictions/resono_week")
import prediction_model_helpers as h  # Universal predictions
import resono_week_predictions as resono_pred  # Resono 1 week model specific

import importlib  # For when coding

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/spark-3.0.3-bin-without-hadoop/jars/slf4j-log4j12-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-3.2.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2021-11-18 18:19:51,501 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Settings

#### Arguments for functions

In [6]:
# file name of historic Resono data (total daily counts)
resono_data_dir = "data/"
file_name = '2021-01-01_2021-11-11_totalsperhour.csv'

In [7]:
# frequency of sampling for data source to predict
freq = 'H'

In [8]:
# how many samples in a day
n_samples_day = 24
# how many samples in a week
n_samples_week = 24*7
# what period to predict for operational forecast (samples)
predict_period = n_samples_week

In [9]:
# list of column name(s) of variabe to predict (can also be "all")
#Y_names = "all" 
#Y_names = ['Albert Cuyp', 'Vondelpark West', 'Rembrandtplein',
#          'Nieuwmarkt', 'Leidseplein', 'Kalverstraat Noord', 'Kalverstraat Zuid']

Y_names = ['Albert Cuyp','Vondelpark West']

# data source (for which the predictions are made)
data_source = 'resono'

# type of prediction (count -> regression or level -> classification)
target = 'count'  # Can only be count 

In [10]:
# input for starting of learnset 
start_learnset = h.get_start_learnset(train_length = 20, date_str = None)

In [11]:
# input for start prediction
start_prediction = "2021-11-15 00:00:00"  # start date of week to predict 
start_prediction = pd.to_datetime(start_prediction)

In [12]:
# Minimum number of training samples needed to make predictions (otherwise no predictions for that location)
min_train_samples = n_samples_week*4

In [13]:
# perform outlier removal ("yes" or "no")
outlier_removal = "no"

In [14]:
# set versions (for storing results)
current_model_version = 'lr_0_0'
current_data_version = "1_0" 

In [15]:
# Report graph settings
report_dir = "/"
week_label = 46
legend = "yes"

### Get predictions

### 1. Prepare data set

In [16]:
t1 = time()

base_df, resono_df, resono_df_raw, start_prediction, end_prediction, Y_names_all = resono_pred.prepare_data(env_az,
                                                                                                            resono_data_dir,
                                                                                                            file_name,
                                                                                                           freq, 
                                                                                                           predict_period, 
                                                                                                           start_prediction,
                                                                                                            n_samples_day, 
                                                                                                           Y_names, 
                                                                                                           target,
                                                                                                           start_learnset)

t2 = time()

print('Completed in %s sec.' % (str(t2 - t1)))

Py4JJavaError: An error occurred while calling o50.load.
: java.nio.file.AccessDeniedException: s3a://knmi-knmi/topics/knmi-observations/2021/06/*/*: getFileStatus on s3a://knmi-knmi/topics/knmi-observations/2021/06/*/*: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 16B8B6FD7E1ED3EE; S3 Extended Request ID: null), S3 Extended Request ID: null:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:230)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:151)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:2275)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:2226)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:2160)
	at org.apache.hadoop.fs.FileSystem.isDirectory(FileSystem.java:1707)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.isDirectory(S3AFileSystem.java

### 2. Make predictions and store in dataframe

In [1]:
# --- remove in version without backtesting
prepared_dfs = dict()
y_scalers = dict()
# ---

# Initialize data frame with predictions
final_df = pd.DataFrame()

gbl = globals()

# Predict for each location
for idx, Y in enumerate(Y_names_all):
    
    # Show location
    print(Y)
    
    # Preprocessed data frame for this location
    preprocessed_df = resono_pred.get_location_df(base_df, resono_df, Y)
    
    # Gather predictons for this location
    prepared_df, predictions, y_scaler = resono_pred.get_resono_predictions(preprocessed_df, resono_df_raw, freq, predict_period, n_samples_day, 
                                                             n_samples_week, Y, data_source, target, 
                                                             outlier_removal, start_learnset,
                                                             current_model_version, current_data_version, 
                                                             start_prediction, end_prediction, min_train_samples)
    # Add predictions to final data frame
    gbl['final_df'+'_'+str(Y)] = pd.concat([final_df, predictions], 0)
    
    # Get and store report figure
    gbl['report_df'+'_'+str(Y)] = resono_pred.get_location_report_df(gbl['final_df'+'_'+str(Y)], prepared_df, y_scaler, Y)
    resono_pred.get_report_plot_hourly(gbl['report_df'+'_'+str(Y)], legend, Y, report_dir, week_label)
    #resono_pred.get_report_plot_daily(report_df, Y, report_dir, week_label)
    
    # --- remove in version without backtesting
    gbl['prepared_dfs'+'_'+str(Y)] = prepared_df
    y_scalers[Y] = y_scaler
    # ---

NameError: name 'pd' is not defined

### 3. Plot prediction graphs

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def get_report_plot_daily(report_df, Y_name, report_dir, week_label):
    '''
    Get a graph of the predictions compared to past observations on a daily rate with week and weekend split.
    '''
    
    # Get daily sum and split between weekdays and weekend
    daily = report_df.resample('D').sum()
    daily_weekend = daily[daily.index.weekday.isin([5, 6])]
    daily_weekdays = daily[daily.index.weekday.isin([0, 1, 2, 3, 4])]
    
    prvs_df = preprocessed_df[start_prediction.date() - timedelta(days=21) : start_prediction.date() - timedelta(hours=23)].resample('D').sum()
    prvs_df['week'] = prvs_df.index.week
    
    prvs_weekend = prvs_df[prvs_df.index.weekday.isin([5, 6])]
    prvs_weekdays = prvs_df[prvs_df.index.weekday.isin([0, 1, 2, 3, 4])]
    
    prvs_weekend['week'] = prvs_weekend.index.week
    prvs_weekend = prvs_weekend.groupby('week').median().reset_index()
    
    prvs_weekdays['week'] = prvs_weekdays.index.week
    prvs_weekdays = prvs_weekdays.groupby('week').median().reset_index()
    
    prvs_weekend = prvs_weekend[['week', Y_name]].rename(columns={Y_name:'Count'})
    prvs_weekdays = prvs_weekdays[['week', Y_name]].rename(columns={Y_name:'Count'})
    
    # Get the mean for weekdays/weekend
    daily_weekdays_avg = daily_weekdays.mean(axis = 0).to_frame().reset_index()
    daily_weekend_avg = daily_weekend.mean(axis = 0).to_frame().reset_index()
    
    # Rename columns
    daily_weekdays_avg.columns = ['week', Y_name]
    daily_weekend_avg.columns = ['week', Y_name]
    
    daily_weekdays_avg = daily_weekdays_avg.rename(columns={Y_name:'Count'})
    daily_weekend_avg = daily_weekend_avg.rename(columns={Y_name:'Count'})
    
    daily_weekdays = pd.concat([prvs_weekdays, daily_weekdays_avg])
    daily_weekends = pd.concat([prvs_weekend, daily_weekend_avg])
    
    daily_weekdays = daily_weekdays.replace({'Voorspelling':week_label})
    daily_weekends = daily_weekends.replace({'Voorspelling':week_label})
    
    daily_weekdays['week'] = daily_weekdays.week.astype(int)
    daily_weekends['week'] = daily_weekends.week.astype(int)
    
    # Determine colors in graph
    daily_weekdays['color'] = 'green'
    daily_weekdays['color'][daily_weekdays['week'] == week_label] = 'lightgreen'
    daily_weekends['color'] = 'green'
    daily_weekends['color'][daily_weekends['week'] == week_label] = 'lightgreen'
    
    # Convert average past 4 weeks to week numbers and prediction to current week number
    #daily_weekdays_avg['Week'] = [str(int(week_label)-3) + "-" + str(int(week_label)-1), week_label]
    #daily_weekend_avg['Week'] = [str(int(week_label)-3) + "-" + str(int(week_label)-1), week_label]
    
    # Parameters
    plt.rcParams["axes.axisbelow"] = True
    plt.rcParams.update({'axes.titlesize': 14,
                     'axes.labelsize': 14, 'xtick.labelsize': 14, 'ytick.labelsize': 14,
                     'axes.labelpad': 8.0
                    })
    
    # Initialize figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (8,4), dpi = 100, frameon = False, sharey = True, 
                                     constrained_layout = True)

    # Title
    fig.suptitle(Y_name, fontsize = 18)
    
    # Weekdays figure
    ax1.bar(x = daily_weekdays['week'], height = daily_weekdays['Count'], color = daily_weekdays['color'])
    ax1.set_title('Doordeweeks (ma-vr)')
    ax1.set_xlabel("Week")
    ax1.set_ylabel("Aantal Resono counts per dag")
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax1.grid(axis = 'y', color = 'lightgrey')

    # Weekend figure
    ax2.bar(x =  daily_weekends['week'], height = daily_weekends['Count'], color = daily_weekends['color'])
    ax2.set_title('Weekend (za-zo)')
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.grid(axis = 'y', color = 'lightgrey')
    ax2.set_xlabel("Week")

    # Store figure
    #os.makedirs(report_dir + week_label, exist_ok=True) 
    #os.chdir(report_dir + week_label)
    fig_name = Y_name.replace(" ", "_") + "_resono_week_ahead_prediction_daily.png"
    plt.savefig(fig_name, bbox_inches='tight', dpi = 500)

In [ ]:
get_report_plot_daily(report_df, 'Albert Cuyp' , report_dir, week_label)

### 4. Evaluate - Check operational prediction

In [ ]:
final_df

### Backtesting --- remove code blocks below in version without backtesting

Test model predictions for the selected location (argument at the beginning) and time period (start_test; within the time period for which the data has been prepared)

In [ ]:
# Input for backtesting

# Start testing from this timestamp until the most recent time slot
start_test = "2021-11-01 00:00:00"
# What period to predict for backtesting (samples)
predict_period = n_samples_week*7

In [ ]:
# If using a NN/LSTM model, it is necessary to also install these libraries
# Related functions have to be uncommented in prediction_model_helpers.py
#pip install keras
#pip install tensorflow

In [ ]:
prepared_dfs

In [ ]:
# Perform backtesting

# Store results
locations = []
rmse_benchmarks = []
rmse_models = []
figs_pred_time = dict()
feat_imps = dict()
figs_feat_imp = dict()

# Predict for each location
for idx, Y in enumerate(Y_names_all):
    
    # Show location
    print(Y)
    
    # Prepare data
    if Y in prepared_dfs:
        df_y_predict_bt, df_y_train_bt, df_y_ground_truth_bt, df_y_ground_truth_bt_scaled, df_X_train_bt, df_X_predict_bt = h.prepare_backtesting(start_test, predict_period, freq, 
                                                                                   prepared_dfs[Y], Y, 
                                                                                   n_samples_week, target, y_scalers[Y])
    
    # Do not perform backtesting if there is not enough training data 
    if (df_X_train_bt.empty) | (len(df_X_train_bt) < min_train_samples):
        print("Not enough training data: no backtesting performed.")
        continue
    
    # Benchmark predictions
    df_y_benchmark = df_y_predict_bt.copy()
    df_y_benchmark[Y] = h.test_model_past_week_bt(df_y_train_bt, df_y_predict_bt, df_y_ground_truth_bt_scaled, 
                                                    predict_period, 
                                                   n_samples_week, target)
    if target == "count":
        df_y_benchmark = h.unscale_y(df_y_benchmark, y_scalers[Y])
        
    error_metrics_benchmark = h.evaluate(df_y_benchmark, df_y_ground_truth_bt, target, Y_name = Y,
                                         print_metrics = False)
    
    rmse_benchmarks.append(error_metrics_benchmark['rmse'])
    
    # Model predictions
    df_y_model = df_y_predict_bt.copy()
    
    model = h.train_model_ridge_regression(df_X_train_bt, df_y_train_bt, Y, target)
    df_y_model[Y] = h.test_model_ridge_regression(model, df_X_predict_bt)
    if target == "count":
        df_y_model = h.unscale_y(df_y_model, y_scalers[Y])
    error_metrics_model = h.evaluate(df_y_model, df_y_ground_truth_bt, target, Y_name = Y, print_metrics = False)
    
    rmse_models.append(error_metrics_model['rmse'])
    
    # Visualize backtesting result
    fig_pred_time = h.visualize_backtesting(df_y_ground_truth_bt, df_y_benchmark, df_y_model, target, Y, 
                                        error_metrics_model, y_label = "Total visitor count", count_to_level = False)
    figs_pred_time[Y] = fig_pred_time
    
    # Feature importance
    feat_imp, fig_feat_imp = h.feature_importance(model.coef_[0], list(df_X_train_bt.columns))
    feat_imps[Y] = feat_imp
    figs_feat_imp[Y] = fig_feat_imp
    
    locations.append(Y)

In [ ]:
# Backtesting results for all locations
df_results = h.backtesting_results_all_locations(locations, rmse_models, rmse_benchmarks)

In [ ]:
# Summarized results
df_results.describe()

In [ ]:
# Locations for which the model performs better
df_results[df_results['RMSE_difference'] < 0]

In [ ]:
# Locations for which the benchmark model performs better
df_results[df_results['RMSE_difference'] > 0]

#### Query results for specific location

In [ ]:
df_results[df_results['Location'] == "Albert Cuyp"]

In [ ]:
figs_pred_time["Vondelpark West"]

In [ ]:
figs_feat_imp["Vondelpark West"]